In [2]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 37.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 74.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 162.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 149.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 48.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 96.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 113.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 85.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 88.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 94.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# !pip install h5py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 68.8 MB/s eta 0:00:00


In [2]:
#dataloading and preprocessing
import h5py
with h5py.File('tactmat.h5', 'r') as dataset:
    samples = dataset['samples'][:]
    materials = dataset['materials'][:]
    materials = [m.decode('utf-8') for m in materials]
train_samples = samples[:, :80,...]  # (36, 80, 1000, 4, 4)
test_samples = samples[:, 80:,...]  # (36, 20, 1000, 4, 4)

# Flatten data
train_samples = train_samples.reshape((36*80, 1000, 16))  # (2880, 1000, 16)
test_samples = test_samples.reshape((36*20, 1000, 16))  # (720, 1000, 16)

# Create labels
train_labels = np.repeat(np.arange(36), 80)  # (2880,)
test_labels = np.repeat(np.arange(36), 20)  # (720,)

# Shuffle
indices = torch.randperm(len(train_samples))
train_samples = train_samples[indices]
train_labels = train_labels[indices]

# One-hot encoding not needed for pytorch CrossEntropyLoss
train_samples = torch.tensor(train_samples.reshape(2880, 1, 1000, 16), dtype=torch.float32)
test_samples = torch.tensor(test_samples.reshape(720, 1, 1000, 16), dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)
test_labels = torch.tensor(test_labels, dtype=torch.long)

In [6]:
# Model Definition
class MCTestTacNet(nn.Module):
    def __init__(self):
        super(MCTestTacNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(15, 5))
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(kernel_size=(10, 1), stride=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(15, 5))
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(kernel_size=(10, 1), stride=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(15, 5))
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(kernel_size=(10, 1), stride=1)
        self.dropout = nn.Dropout(0.8)

        # Dummy forward pass to calculate the flattened size
        self.flattened_size = self._get_flatten_size()

        self.fc1 = nn.Linear(self.flattened_size, 512)
        self.bn4 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 36)

    def _get_flatten_size(self): # helper function for dynamic sizing of linear layer
        with torch.no_grad():
            x = torch.zeros(1, 1, 1000, 16)  # example
            x = self.pool3(self.bn3(self.conv3(self.pool2(self.bn2(self.conv2(self.pool1(self.bn1(self.conv1(x)))))))))
            return x.view(1, -1).size(1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool1(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool2(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.pool3(x)
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.bn4(self.fc1(self.dropout(x))))
        x = self.fc2(x)  # no softmax bc cross entropy loss includes it
        return x

# Instantiate Model
model = MCTestTacNet()
model.to(device)


# Training func
def train_model(model, train_samples, train_labels,val_samples,val_labels,  num_epochs=10, batch_size=32, learning_rate=0.001):
    train_dataset = TensorDataset(train_samples, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    
    val_dataset = TensorDataset(val_samples, val_labels)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)    
    
    train_samples = train_samples.to(device)
    train_labels = train_labels.to(device)
    
    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=170, gamma=0.1)
    
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0

        for inputs, labels in tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]", unit="batch"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs) #forward pass
            loss = criterion(outputs, labels) #loss
            loss.backward() #backward pass
            optimizer.step() #update params
            running_loss += loss.item() #calc running loss

        average_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss:.4f}")
        
        scheduler.step()
        
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    val_loss /= len(val_loader)
    print(f"Validation Loss: {val_loss:.4f}")

    return model, val_loss
    
    print("Training complete!")

#run training
trained_model, val_error = train_model(model, train_samples, train_labels, test_samples, test_labels, num_epochs=200, batch_size=32, learning_rate=0.001)

# Save the model and validation error
torch.save(trained_model.state_dict(), "trained_model.pth")
with open("val_error.txt", "w") as f:
    f.write(str(val_error))

Epoch [1/200]: 100%|██████████| 90/90 [00:23<00:00,  3.77batch/s]


Epoch [1/200], Loss: 2.6790


Epoch [2/200]: 100%|██████████| 90/90 [00:24<00:00,  3.66batch/s]


Epoch [2/200], Loss: 1.6281


Epoch [3/200]: 100%|██████████| 90/90 [00:24<00:00,  3.64batch/s]


...

Epoch [198/200], Loss: 0.0006


Epoch [199/200]: 100%|██████████| 90/90 [00:24<00:00,  3.68batch/s]


Epoch [199/200], Loss: 0.0005


Epoch [200/200]: 100%|██████████| 90/90 [00:24<00:00,  3.68batch/s]


Epoch [200/200], Loss: 0.0007
Validation Loss: 1.8025


In [7]:
# Save val_samples
torch.save(test_samples, "test_samples.pth")

# Save val_dataset
torch.save(test_labels, "test_labels.pth")
